# Evaluate MCQs generated by Llama

In this notebook, we will evaluate the MCQs using the evaluation in src/mcq_evaluator.py. This evaluation will serve as a selection basis for the training and test sets that will be used later in the fine-tuning of the Phi2 model.

## Imports

In [1]:
import sys
from pathlib import Path

# Add the root directory to sys.path
root_path = Path().resolve().parent
sys.path.append(str(root_path))

%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import pandas as pd
from tqdm import tqdm
from src.mcq_evaluator import MCQEvaluator

/home/iradukum/Documents/projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Evaluate MCQs

In [5]:
MCQ_PATH = "../data/synthetic_contexts/generated_mcqs_llama_1000.jsonl"
OUTPUT_CSV = "../data/evaluation_scores/mcq_eval_scores_1000.csv"

In [6]:
# Load MCQs
with open(MCQ_PATH, 'r') as f:
    lines = [json.loads(l) for l in f.readlines()]

In [7]:
# Evaluate
evaluator = MCQEvaluator()
eval_data = []

for item in tqdm(lines):
    scores = evaluator.evaluate(
        mcq={
            "question": item["question"],
            "options": item["options"],
            "answer": item["answer"],
            "explanation": item["explanation"]
        },
        summary=item["summary"],
        objective=item["objective"]
    )
    row = {**item, **scores}
    eval_data.append(row)

100%|███████████████████████████████████████| 1000/1000 [02:39<00:00,  6.26it/s]


In [8]:
eval_df = pd.DataFrame(eval_data)
eval_df

,focus,qtypes,summary,objective,question,options,answer,explanation,relevance_to_summary,alignement_with_objective,plausibility_of_distractors,plausibility_qa,medical_validity,average
0,Diphtheria,[information],Diphtheria is a serious bacterial infection ca...,Describe the importance of timely vaccination ...,A patient presents with symptoms of diphtheria...,{'A': 'The patient's immunity is likely to pro...,D,Given the patient's vaccination history and th...,0.708517,0.736968,0.833,1.0,1.0,0.856
1,trichothiodystrophy,"[frequency, genetic changes, information, trea...","Trichothiodystrophy is a rare, inherited condi...",Analyze the inheritance pattern of a rare gene...,A patient is diagnosed with Trichothiodystroph...,"{'A': 'Autosomal dominant pattern, where one m...",C,Trichothiodystrophy is inherited in an autosom...,0.792817,0.544787,0.750,1.0,1.0,0.818
2,Nemaline myopathy,[information],Nemaline myopathy is a genetic disorder that p...,Identify the primary inheritance pattern of Ne...,What is the typical genetic inheritance patter...,"{'A': 'Autosomal Dominant', 'B': 'X-linked Rec...",D,Nemaline Myopathy is primarily inherited in an...,0.801020,0.921066,0.833,1.0,1.0,0.911
3,Gracile bone dysplasia,[symptoms],Gracile bone dysplasia is a rare genetic disor...,Identify the primary cause of respiratory comp...,What is the primary reason why individuals wit...,{'A': 'Cardiac anomalies leading to pulmonary ...,C,Individuals with Gracile bone dysplasia are at...,0.683419,0.878695,0.917,1.0,1.0,0.896
4,Gastrocutaneous syndrome,[symptoms],Gastrocutaneous syndrome is a complex conditio...,Identify a characteristic symptom associated w...,Which of the following skin lesions is commonl...,"{'A': 'Pityriasis alba-like macules', 'B': 'Me...",B,Melanocytic nevi are a characteristic skin les...,0.678607,0.729024,1.000,1.0,1.0,0.882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Cataract,"[susceptibility, symptoms, treatment, informat...",A cataract is a clouding of the lens in the ey...,Analyze the relationship between environmental...,A patient with a history of prolonged exposure...,"{'A': 'Macular degeneration', 'B': 'Cataract',...",B,Prolonged exposure to ultraviolet sunlight has...,0.474832,0.507629,1.000,1.0,0.5,0.696
996,Pericarditis,"[information, causes, susceptibility, symptoms...",Pericarditis is a condition characterized by i...,Evaluate the appropriate initial treatment app...,A 35-year-old man presents with sudden onset o...,{'A': 'Order an emergency echocardiogram to ev...,B,Colchicine and prednisone are anti-inflammator...,0.579525,0.427334,0.833,1.0,1.0,0.768
997,Ectopic Pregnancy,[information],An ectopic pregnancy occurs when a fertilized ...,Identify the typical location where an ectopic...,Where does an ectopic pregnancy most commonly ...,"{'A': 'Ovaries', 'B': 'Fallopian tubes', 'C': ...",B,"The correct answer is B, as fallopian tubes ar...",0.626850,0.858277,1.000,1.0,0.5,0.797
998,15q24 microdeletion,"[frequency, genetic changes, information, trea...",15q24 microdeletion is a rare chromosomal diso...,Evaluate the relationship between a genetic co...,A patient diagnosed with 15q24 microdeletion p...,{'A': 'The deletion of 1-6 million DNA base pa...,B,Individuals with 15q24 microdeletion are at ri...,0.737288,0.625763,1.000,1.0,1.0,0.873


In [9]:
os.makedirs("../data/evaluation_scores", exist_ok=True)
eval_df.to_csv(OUTPUT_CSV, index=False)

print(f"Evaluated {len(eval_data)} MCQs. Evaluation results in {OUTPUT_CSV}")

Evaluated 1000 MCQs → ../data/evaluation_scores/mcq_eval_scores_1000.csv
